In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import json
import types
from os import listdir
from os.path import isfile, join
from unidecode import unidecode
import pandas as pd
import codecs

In [2]:
LYRICSSITE = 'http://www.lyrics.wikia.com/api.php'
DATA_PATH = '../data/'
BILLBOARD_RAW_PATH = 'billboard_raw/'
BILLBOARD_RAW_PATH2 = 'billboard_raw2/'
BILLBOARD_LYRIC_PATH = 'billboard_lyrics/'
BILLBOARD_LYRIC_PATH2  = 'billboard_lyrics2/'
NUMBER, ARTIST, SONG = range(3)

In [3]:
def replace_br(lyrics_soup):
    lyric = ''
    for br in lyrics_soup.findAll('br'):
        next_s = br.next_sibling
        if next_s:
            next2_s = next_s.next_sibling
        text = str(next_s).strip()
        if "<" in text:
            pass
        else:
            lyric = " ".join([lyric,text])
    return lyric

In [49]:
def download_lyrics(y,df):

    position_list = df['position'].tolist()
    artist_list = df['artist'].tolist()
    title_list = df['title'].tolist()
    lyric_list = []
    
    for i in range(len(artist_list)):
        lyric_list.append("no lyrics") 
        params = {
            'action': 'lyrics',
            'artist': artist_list[i],
            'song': title_list[i]
        }
        link_r = requests.get(LYRICSSITE, params = params)
        link_soup = BeautifulSoup(link_r.text, 'html.parser')
        link = 0
        anc = link_soup.a
        if link_soup.a:
            if link_soup.a.has_attr('href'):
                link = anc['href']
        if link != 0:
            lyrics_r = requests.get(link)
            lyrics_soup = BeautifulSoup(lyrics_r.text, 'html.parser')
            lyrics_divs = lyrics_soup.findAll('div', { 'class': 'lyricbox'})

            for div in lyrics_divs:
                lyric = replace_br(lyrics_soup)
                if not lyric:
                    print('problem with lyric')
                    continue
                ###DATA CLEANING: filter was a 
                lyric = lyric.replace(' was a ', '')
                lyric_list[-1] = lyric
        #Reference: https://github.com/alexmwu/hot100
        else:
            ###DATA CLEANING: no lyrics
            lyric_list.append('no lyrics')
            print('no lyric')

    df['lyric']= lyric_list
    df = df[['index','position','year','title','artist','lyric','decade']]
    return df


In [50]:
start_year = 2006
curr_year = date.today().year

for y in range(start_year, curr_year):
    file = str(y) + 'hot100.csv'
    fname = DATA_PATH+BILLBOARD_RAW_PATH+file
    print(fname)
    #loading raw billbaord file
    df = pd.read_csv(DATA_PATH+BILLBOARD_RAW_PATH+file,sep='@')
    df = download_lyrics(y,df)
    display(df)
    df.to_csv(DATA_PATH+BILLBOARD_LYRIC_PATH+file, sep='@') 



../data/billboard_raw/2006hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,2006,Bad Day,Daniel Powter,"August 23, 2009You kick up the leaves and the...",2000
1,1,2,2006,Temperature,Sean Paul,"So me give it to, so me give to, so me give it...",2000
2,2,3,2006,Promiscuous,Nelly Furtado Featuring Timbaland,Timbaland: Nope Nelly: Didn't think so Timbala...,2000
3,3,4,2006,You're Beautiful,James Blunt,"February 11, 2007My life is brilliant My love...",2000
4,4,5,2006,Hips Don't Lie,Shakira Featuring Wyclef Jean,"Ladies up in here tonight No fighting, no figh...",2000
5,5,6,2006,Unwritten,Natasha Bedingfield,"April 05, 2007Can't read my mind I'm undefine...",2000
6,6,7,2006,Crazy,Gnarls Barkley,"I remember, I remember when I lost my mind The...",2000
7,7,8,2006,Ridin',Chamillionaire Featuring Krayzie Bone,"August 10, 2006Patrollin' and tryin' to catch...",2000
8,8,9,2006,SexyBack,Justin Timberlake,Them other boys don't know how to act I think ...,2000
9,9,10,2006,Check On It,Beyonce Featuring Slim Thug,With all them clowns and the wangstas Good gir...,2000


../data/billboard_raw/2007hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,2007,Irreplaceable,Beyonce,"January 01, 2007To the left, to the left, mmm...",2000
1,1,2,2007,Umbrella,Rihanna Featuring Jay-Z,Uh-huh uh-huh (Good girl gone bad) Uh-huh uh-h...,2000
2,2,3,2007,The Sweet Escape,Gwen Stefani Featuring Akon,"Woo-hoo, yee-hoo Woo-hoo, yee-hoo Woo-hoo, yee...",2000
3,3,4,2007,Big Girls Don't Cry,Fergie,The smell of your skin lingers on me now You'r...,2000
4,4,5,2007,Buy U A Drank (Shawty Snappin'),T-Pain Featuring Yung Joc,"T-Pain Damn, shawty snap Yung Joc (Shawty) Sha...",2000
5,5,6,2007,Before He Cheats,Carrie Underwood,And she's probably getting frisky Right now he...,2000
6,6,7,2007,Hey There Delilah,Plain White T's,"April 24, 2007What's it like in New York City...",2000
7,7,8,2007,I Wanna Love You,Akon Featuring Snoop Dogg,"Convict, music And you know we up front I see ...",2000
8,8,9,2007,Say It Right,Nelly Furtado,"June 04, 2007Say it right, say it all You eit...",2000
9,9,10,2007,Glamorous,Fergie Featuring Ludacris,If you ain't got no money take yo' broke ass h...,2000


../data/billboard_raw/2008hot100.csv


...

,index,position,year,title,artist,lyric,decade
0,0,1,2015,Uptown Funk!,Mark Ronson Featuring Bruno Mars,"This hit, that ice cold Michelle Pfeiffer, tha...",2010
1,1,2,2015,Thinking Out Loud,Ed Sheeran,"November 30, 2014And I can't sweep you off of...",2010
2,2,3,2015,See You Again,Wiz Khalifa Featuring Charlie Puth,And I'll tell you all about it when I see you ...,2010
3,3,4,2015,Trap Queen,Fetty Wap,"1738 I'm like ""hey, what's up, hello"" Seen yo ...",2010
4,4,5,2015,Sugar,Maroon 5,"I need your loving, loving, I need it now When...",2010
5,5,6,2015,Shut Up And Dance,WALK THE MOON,"Just keep your eyes on me"" I said, ""You're hol...",2010
6,6,7,2015,Blank Space,Taylor Swift,"I could show you incredible things Magic, madn...",2010
7,7,8,2015,Watch Me,Silento,no lyrics,2010
8,8,9,2015,Earned It (Fifty Shades Of Grey),The Weeknd,"'Cause I see nobody, nobody but you, you, you ...",2010
9,9,10,2015,The Hills,The Weeknd,You said keep our business on the low-low I'm ...,2010


../data/billboard_raw/2016hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,2016,Love Yourself,Justin Bieber,And all the clubs you get in using my name You...,2010
1,1,2,2016,Sorry,Justin Bieber,You know I try but I don't do too well with ap...,2010
2,2,3,2016,One Dance,Drake Featuring WizKid & Kyla,"Grips on your waist Front way, back way You kn...",2010
3,3,4,2016,Work,Rihanna Featuring Drake,"You see me I be Work, work, work, work, work, ...",2010
4,4,5,2016,Stressed Out,twenty one pilots,I wish I had a better voice that sang some bet...,2010
5,5,6,2016,Panda,Desiigner,I guess so They been waiting for this shit for...,2010
6,6,7,2016,Hello,Adele,"March 20, 2018 January 22, 2018I was wonderin...",2010
7,7,8,2016,Don't Let Me Down,The Chainsmokers Featuring Daya,"Right now I need a miracle Hurry up now, I nee...",2010
8,8,9,2016,Can't Stop The Feeling!,Justin Timberlake,"It goes electric, wavey when I turn it on All ...",2010
9,9,10,2016,Closer,The Chainsmokers Featuring Halsey,I drink too much and that's an issue but I'm o...,2010


../data/billboard_raw/2017hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,2017,Shape Of You,Ed Sheeran,So the bar is where I go Me and my friends at ...,2010
1,1,2,2017,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bieber,"So thankful for that, it's such a blessin', y...",2010
2,2,3,2017,That's What I Like,Bruno Mars,"Baby girl, what's hatnin'? You and your ass in...",2010
3,3,4,2017,Humble.,Kendrick Lamar,"Even a day for me Way (yeah, yeah) Aye, I reme...",2010
4,4,5,2017,Something Just Like This,The Chainsmokers & Coldplay,The legends and the myths Achilles and his gol...,2010
5,5,6,2017,Bad And Boujee,Migos Featuring Lil Uzi Vert,You know so we never really had no old money W...,2010
6,6,7,2017,Closer,The Chainsmokers Featuring Halsey,I drink too much and that's an issue but I'm o...,2010
7,7,8,2017,Body Like A Back Road,Sam Hunt,"First time I seen her walk by, man I 'bout fel...",2010
8,8,9,2017,Believer,Imagine Dragons,I'mma say all the words inside my head I'm fir...,2010
9,9,10,2017,Congratulations,Post Malone Featuring Quavo,"Yeah-yeah No-nah-nah Yeah My mama called, seen...",2010


In [56]:

start_year = 1970
curr_year = 1977
    
for y in range(start_year, curr_year):
    file = str(y) + 'hot100.csv'
    fname = DATA_PATH+BILLBOARD_RAW_PATH2+file
    print(fname)
    #loading raw billbaord file
    df = pd.read_csv(DATA_PATH+BILLBOARD_RAW_PATH2+file,sep='@')
    df = download_lyrics(y,df)
    display(df)
    df.to_csv(DATA_PATH+BILLBOARD_LYRIC_PATH2+file, sep='@') 


../data/billboard_raw2/1970hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1970,Bridge Over Troubled Water,Simon and Garfunkel,Feeling small When tears are in your eyes I wi...,1970
1,1,2,1970,(They Long To Be) Close To You,Carpenters,Everytime you are near? Just like me they long...,1970
2,2,3,1970,American Woman / No Sugar Tonight,Guess Who,no lyrics,1970
3,3,4,1970,Raindrops Keep Fallin' On My Head,B.J. Thomas,And just like the guy whose feet are too big f...,1970
4,4,5,1970,War,Edwin Starr,"What is it good for? Absolutely nothing, uh-hu...",1970
5,5,6,1970,Ain't No Mountain High Enough,Diana Ross,"August 26, 2007Ooh, ooh Ah, ah, ah Ooh, ooh I...",1970
6,6,7,1970,I'll Be There,Jackson 5,We must bring salvation back Where there is lo...,1970
7,7,8,1970,Get Ready,Rare Earth,"Whenever I'm asked what makes my dreams real, ...",1970
8,8,9,1970,Let It Be,Beatles,Mother Mary comes to me Speaking words of wisd...,1970
9,9,10,1970,Band Of Gold,Freda Payne,All that's left is a band of gold All that's l...,1970


../data/billboard_raw2/1971hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1971,Joy To The World,Three Dog Night,Was a good friend of mine I never understood a...,1970
1,1,2,1971,Maggie May / (Find A) Reason To Believe,Rod Stewart,no lyrics,1970
2,2,3,1971,It's Too Late / I Feel The Earth Move,Carole King,no lyrics,1970
3,3,4,1971,One Bad Apple,Osmonds,I can tell you've been hurt By that look on yo...,1970
4,4,5,1971,How Can You Mend A Broken Heart,Bee Gees,When living for my life Was everything a man c...,1970
5,5,6,1971,Indian Reservation,Raiders,Put us on this reservation Took away our ways ...,1970
6,6,7,1971,Go Away Little Girl,Donny Osmond,Go away little girl I'm not supposed to be alo...,1970
7,7,8,1971,"Take Me Home, Country Roads",John Denver,"October 12, 2010Blue Ridge Mountains, Shenand...",1970
8,8,9,1971,Just My Imagination (Running Away With Me),Temptations,Ooh Each day through my window I watch her as ...,1970
9,9,10,1971,Knock Three Times,Dawn,Dancing alone every night while I live right a...,1970


../data/billboard_raw2/1972hot100.csv
problem with lyric


...

,index,position,year,title,artist,lyric,decade
0,0,1,1974,The Way We Were,Barbra Streisand,Light the corners of my mind Misty watercolor ...,1970
1,1,2,1974,Seasons In The Sun,Terry Jacks,We've known each other since we were nine or t...,1970
2,2,3,1974,Love's Theme,Love Unlimited Orchestra,was a,1970
3,3,4,1974,Come And Get Your Love,Redbone,What's the matter with your hair? Yeah Hail (h...,1970
4,4,5,1974,Dancing Machine,Jackson 5,She's a dancing machine Ah baby Move it baby A...,1970
5,5,6,1974,The Loco-Motion,Grand Funk Railroad,I know you'll get to like it if you give it a ...,1970
6,6,7,1974,TSOP,MFSB,"Let's get it on, it's time to get down Let's g...",1970
7,7,8,1974,The Streak,Ray Stevens,"that is news across the nation, on the scene a...",1970
8,8,9,1974,Bennie And The Jets,Elton John,The spotlight's hitting something That's been ...,1970
9,9,10,1974,One Hell Of A Woman,Mac Davis,When you're perfect in every way. I can't wai...,1970


../data/billboard_raw2/1975hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1975,Love Will Keep Us Together,Captain and Tennille,Think of me babe whenever Some sweet talking g...,1970
1,1,2,1975,Rhinestone Cowboy,Glen Campbell,Singin' the same old song I know every crack i...,1970
2,2,3,1975,Philadelphia Freedom,Elton John,If the cause was right I'd leave to find the a...,1970
3,3,4,1975,Before The Next Teardrop Falls,Freddy Fender,Then I wish you all the best It's your happine...,1970
4,4,5,1975,My Eyes Adored You,Frankie Valli,Though I never laid a hand on you My eyes ador...,1970
5,5,6,1975,Shining Star,"Earth, Wind and Fire",When you wish upon a star Your dreams will tak...,1970
6,6,7,1975,Fame,David Bowie,"Fame lets him loose, hard to swallow Fame puts...",1970
7,7,8,1975,Laughter In The Rain,Neil Sedaka,"It starts to rain, it begins to pour Without a...",1970
8,8,9,1975,One Of These Nights,Eagles,One of these crazy old nights We're gonna find...,1970
9,9,10,1975,Thank God I'm A Country Boy,John Denver,Ain't much an old country boy like me can't ha...,1970


../data/billboard_raw2/1976hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1976,Silly Love Songs,Wings,But look around me and I see it isn't so Some ...,1970
1,1,2,1976,Don't Go Breaking My Heart,Elton John and Kiki Dee,no lyrics,1970
2,2,3,1976,Disco Lady,Johnnie Taylor,"Move it in, move it 'round, disco lady Move it...",1970
3,3,4,1976,"December, 1963 (Oh, What A Night)",Four Seasons,"Late December, back in '63 What a very special...",1970
4,4,5,1976,Play That Funky Music,Wild Cherry,"Yeah, hey Hey-ah, once Iboogie singer Playing ...",1970
5,5,6,1976,Kiss And Say Goodbye,Manhattans,I called you here today for a bit of bad news ...,1970
6,6,7,1976,Love Machine (Part 1),The Miracles,I'm just a love machine And I won't work for n...,1970
7,7,8,1976,50 Ways To Leave Your Lover,Paul Simon,"""The answer is easy if you take it logically I...",1970
8,8,9,1976,Love Is Alive,Gary Wright,To realize just what I have found I have lived...,1970
9,9,10,1976,A Fifth Of Beethoven,Walter Murphy and The Big Apple Band,no lyrics,1970


In [53]:

start_year = 1977
curr_year = 1990
    
for y in range(start_year, curr_year):
    file = str(y) + 'hot100.csv'
    fname = DATA_PATH+BILLBOARD_RAW_PATH+file
    print(fname)
    #loading raw billbaord file
    df = pd.read_csv(DATA_PATH+BILLBOARD_RAW_PATH2+file,sep='@')
    df = download_lyrics(y,df)
    display(df)
    df.to_csv(DATA_PATH+BILLBOARD_LYRIC_PATH2+file, sep='@') 


../data/billboard_raw/1977hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1977,Tonight's The Night (Gonna Be Alright),Rod Stewart,Stay away from my back door too Disconnect the...,1970
1,1,2,1977,I Just Want To Be Your Everything,Andy Gibb,You and me been finding each other for so long...,1970
2,2,3,1977,Best Of My Love,Emotions,"And make me smile with glee Never, never will ...",1970
3,3,4,1977,"Love Theme From ""A Star Is Born""",Barbra Streisand,no lyrics,1970
4,4,5,1977,Angel In Your Arms,Hot,To believe everything you say is true That jus...,1970
5,5,6,1977,I Like Dreamin',Kenny Nolan,"I like dreamin', closing my eyes and feeling f...",1970
6,6,7,1977,Don't Leave Me This Way,Thelma Houston,"I can't survive, I can't stay alive Without yo...",1970
7,7,8,1977,(Your Love Has Lifted Me) Higher And Higher,Rita Coolidge,Than I've ever been lifted before Your love is...,1970
8,8,9,1977,Undercover Angel,Alan O'Day,Then I heard a voice beside me and she softly ...,1970
9,9,10,1977,Torn Between Two Lovers,Mary MacGregor,Even though she knows how much it's gonna hurt...,1970


../data/billboard_raw/1978hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1978,Shadow Dancing,Andy Gibb,"I was chasing your direction, I was telling yo...",1970
1,1,2,1978,Night Fever,Bee Gees,There is movement all around There is somethin...,1970
2,2,3,1978,You Light Up My Life,Debby Boone,Waiting for someone to sing me his song So man...,1970
3,3,4,1978,Stayin' Alive,Bee Gees,"I'm a woman's man, no time to talk Music loud ...",1970
4,4,5,1978,Kiss You All Over,Exile,Gonna light your fire All day I've been thinki...,1970
5,5,6,1978,How Deep Is Your Love,Bee Gees,I know your eyes in the morning sun I feel you...,1970
6,6,7,1978,Baby Come Back,Player,Doing anything just to get you off of my mind ...,1970
7,7,8,1978,(Love Is) Thicker Than Water,Andy Gibb,Love is thicker than water You are this dreame...,1970
8,8,9,1978,Boogie Oogie Oogie,A Taste Of Honey,"Boy oh boy, have I got news for you Everybody ...",1970
9,9,10,1978,Three Times A Lady,Commodores,The memories are all in my mind And now that w...,1970


../data/billboard_raw/1979hot100.csv


...

,index,position,year,title,artist,lyric,decade
0,0,1,1987,Walk Like An Egyptian,Bangles,"They do the sand dance, don't you know? If the...",1980
1,1,2,1987,Alone,Heart,"September 22, 2006I'm lying here, the room's ...",1980
2,2,3,1987,Shake You Down,Gregory Abbott,"Girl, I've been watching you From so far acros...",1980
3,3,4,1987,I Wanna Dance With Somebody (Who Loves Me),Whitney Houston,And the sun begins to fade Still enough time t...,1980
4,4,5,1987,Nothing's Gonna Stop Us Now,Starship,I see a paradise This world that I found Is to...,1980
5,5,6,1987,C'est La Vie,Robbie Nevil,But they got me working night and day Punching...,1980
6,6,7,1987,Here I Go Again,Whitesnake,"February 10, 2010But I sure know where I've b...",1980
7,7,8,1987,The Way It Is,Bruce Hornsby and The Range,"January 16, 2008Waiting for the welfare dime ...",1980
8,8,9,1987,Shakedown,Bob Seger,You'll find it's not enough No matter who you ...,1980
9,9,10,1987,Livin' On A Prayer,Bon Jovi,"January 23, 2009 November 17, 2012Not so long...",1980


../data/billboard_raw/1988hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1988,Faith,George Michael,If I could touch your body I know not everybod...,1980
1,1,2,1988,Need You Tonight,INXS,All you've got is this moment 21st century is ...,1980
2,2,3,1988,Got My Mind Set On You,George Harrison,"August 06, 2007I got my mind set on you I got...",1980
3,3,4,1988,Never Gonna Give You Up,Rick Astley,"October 02, 2007We're no strangers to love Yo...",1980
4,4,5,1988,Sweet Child O' Mine,Guns N' Roses,"December 30, 2007Reminds me of childhood memo...",1980
5,5,6,1988,So Emotional,Whitney Houston,"I just do Ooh, ooh I've been hearing your hear...",1980
6,6,7,1988,Heaven Is A Place On Earth,Belinda Carlisle,"Ooh, heaven is a place on earth They say in he...",1980
7,7,8,1988,Could've Been,Tiffany,Are just about to die When I think about what ...,1980
8,8,9,1988,Hands To Heaven,Breathe,"November 26, 2007There's so much tenderness i...",1980
9,9,10,1988,Roll With It,Steve Winwood,"Don't stop and lose your touch, oh no, baby Ha...",1980


../data/billboard_raw/1989hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1989,Look Away,Chicago,"January 10, 2008Told me 'bout the new love yo...",1980
1,1,2,1989,My Prerogative,Bobby Brown,Everybody's talking all this stuff about me Wh...,1980
2,2,3,1989,Every Rose Has Its Thorn,Poison,"June 15, 2009Although we both lie close toget...",1980
3,3,4,1989,Straight Up,Paula Abdul,I don't know which way to go If you are all th...,1980
4,4,5,1989,Miss You Much,Janet Jackson,Like an arrow through my heart That's the pain...,1980
5,5,6,1989,Cold Hearted,Paula Abdul,"Look into his eyes Oh, oh, he's been telling l...",1980
6,6,7,1989,Wind Beneath My Wings,Bette Midler,To never have sunlight on your face You were c...,1980
7,7,8,1989,Girl You Know Its True,Milli Vanilli,no lyrics,1980
8,8,9,1989,"Baby, I Love Your Way-Freebird",Will To Power,no lyrics,1980
9,9,10,1989,Giving You The Best That I Got,Anita Baker,In exchange for everything you give to me Rea...,1980


In [54]:

start_year = 1990
curr_year = 2000
    
for y in range(start_year, curr_year):
    file = str(y) + 'hot100.csv'
    fname = DATA_PATH+BILLBOARD_RAW_PATH+file
    print(fname)
    #loading raw billbaord file
    df = pd.read_csv(DATA_PATH+BILLBOARD_RAW_PATH2+file,sep='@')
    df = download_lyrics(y,df)
    display(df)
    df.to_csv(DATA_PATH+BILLBOARD_LYRIC_PATH2+file, sep='@') 


../data/billboard_raw/1990hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1990,Hold On,Wilson Phillips,Why do you lock yourself up in these chains? N...,1990
1,1,2,1990,It Must Have Been Love,Roxette,English:But it's over now Lay a whisper on my...,1990
2,2,3,1990,Nothing Compares 2 U,Sinead O'Connor,no lyrics,1990
3,3,4,1990,Poison,Bell Biv Devoe,I sense something strange in my mind Situation...,1990
4,4,5,1990,Vogue,Madonna,"Strike a pose Strike a pose Vogue, vogue, vogu...",1990
5,5,6,1990,Vision Of Love,Mariah Carey,Sweet destiny Carried me through desperation T...,1990
6,6,7,1990,Another Day In Paradise,Phil Collins,"""Sir, can you help me? It's cold and I've nowh...",1990
7,7,8,1990,Hold On,En Vogue,"I treated you bad Wrong my dear But since, si...",1990
8,8,9,1990,Cradle Of Love,Billy Idol,"Rock the cradle of love Yes, the cradle of lov...",1990
9,9,10,1990,Blaze Of Glory,Jon Bon Jovi,And I raise my weary head I've got an old coat...,1990


../data/billboard_raw/1991hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1991,(Everything I Do) I Do It For You,Bryan Adams,"What you mean to me Search your heart, search ...",1990
1,1,2,1991,I Wanna Sex You Up,Color Me Badd,"Come inside, take off your coat, I'll make you...",1990
2,2,3,1991,Gonna Make You Sweat,C+C Music Factory,(Everybody dance now!) (Give me the music Give...,1990
3,3,4,1991,Rush Rush,Paula Abdul,"Ooh, na na na You're the whisper of a summer b...",1990
4,4,5,1991,One More Try,Timmy T,I didn't mean to make you cry I didn't mean to...,1990
5,5,6,1991,Unbelievable,EMF,(What the fuck was that?) You burden me with y...,1990
6,6,7,1991,More Than Words,Extreme,"April 14, 2013Is not the words I want to hear...",1990
7,7,8,1991,I Like The Way (The Kissing Game),Hi-Five,"Oh, ooh, ooh, ooh, ooh I like the way I, I lik...",1990
8,8,9,1991,The First Time,Surface,We shared together holding hands And walking i...,1990
9,9,10,1991,"Baby, Baby",Amy Grant,no lyrics,1990


../data/billboard_raw/1992hot100.csv


...

,index,position,year,title,artist,lyric,decade
0,0,1,1997,Candle In The Wind 1997,Elton John,"August 31, 2011May you ever grow in our heart...",1990
1,1,2,1997,Foolish Games / You Were Meant For Me,Jewel,no lyrics,1990
2,2,3,1997,I'll Be Missing You,Puff Daddy and Faith Evans,"October 30, 2012I hope I'm dreaming I can't b...",1990
3,3,4,1997,Un-Break My Heart,Toni Braxton,Don't leave me out in the rain Come back and b...,1990
4,4,5,1997,Can't Nobody Hold Me Down,Puff Daddy,"Now with Sean on the hot track, melt like it's...",1990
5,5,6,1997,I Believe I Can Fly,R. Kelly,"February 17, 2011And life was nothing but an ...",1990
6,6,7,1997,Don't Let Go (Love),En Vogue,Don't you want to be more than friends Hold me...,1990
7,7,8,1997,Return Of The Mack,Mark Morrison,"Well I tried to tell you so (yes, I did) But I...",1990
8,8,9,1997,How Do I Live,LeAnn Rimes,Get through one night without you? If I had t...,1990
9,9,10,1997,Wannabe,Spice Girls,"So tell me what you want, what you really, rea...",1990


../data/billboard_raw/1998hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1998,Too Close,Next,"Hmm, yeah come on Dance with me now, haha Oh, ...",1990
1,1,2,1998,The Boy Is Mine,Brandy and Monica,no lyrics,1990
2,2,3,1998,You're Still The One,Shania Twain,"February 14, 2011Look how far we've come my b...",1990
3,3,4,1998,Truly Madly Deeply,Savage Garden,"I'll be your fantasy I'll be your hope, I'll b...",1990
4,4,5,1998,How Do I Live,LeAnn Rimes,Get through one night without you? If I had t...,1990
5,5,6,1998,Together Again,Janet,There are times when I feel your love around m...,1990
6,6,7,1998,All My Life,K-Ci and JoJo,no lyrics,1990
7,7,8,1998,Candle In The Wind 1997,Elton John,"August 31, 2011May you ever grow in our heart...",1990
8,8,9,1998,Nice & Slow,Usher,Really? You know I'm coming over right? Now yo...,1990
9,9,10,1998,I Don't Want To Wait,Paula Cole,And say a little prayer for right You know tha...,1990


../data/billboard_raw/1999hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,1999,Believe,Cher,"After love, after love, after love, after love...",1990
1,1,2,1999,No Scrubs,TLC,"May 04, 2012And is also known as a Busta Alwa...",1990
2,2,3,1999,Angel Of Mine,Monica,There was something inside of you Something I ...,1990
3,3,4,1999,Heartbreak Hotel,Whitney Houston,This is the heartbreak hotel This is the heart...,1990
4,4,5,1999,...Baby One More Time,Britney Spears,"Oh, baby, baby Oh, baby, baby, how was I suppo...",1990
5,5,6,1999,Kiss Me,Sixpence None The Richer,"Nightly, beside the green, green grass Swing, ...",1990
6,6,7,1999,Genie In A Bottle,Christina Aguilera,For a century of lonely nights Waiting for som...,1990
7,7,8,1999,Every Morning,Sugar Ray,From the corner of my girlfriend's four-post b...,1990
8,8,9,1999,Nobody's Supposed To Be Here,Deborah Cox,Nobody's supposed to be here I've tried that l...,1990
9,9,10,1999,Livin' La Vida Loca,Ricky Martin,English:Black cats and voodoo dolls I feel a ...,1990


In [55]:

start_year = 2000
curr_year = 2007
    
for y in range(start_year, curr_year):
    file = str(y) + 'hot100.csv'
    fname = DATA_PATH+BILLBOARD_RAW_PATH+file
    print(fname)
    #loading raw billbaord file
    df = pd.read_csv(DATA_PATH+BILLBOARD_RAW_PATH2+file,sep='@')
    df = download_lyrics(y,df)
    display(df)
    df.to_csv(DATA_PATH+BILLBOARD_LYRIC_PATH2+file, sep='@') 


../data/billboard_raw/2000hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,2000,Breathe,Faith Hill,Being with you gets me that way I watch the su...,2000
1,1,2,2000,Smooth,Santana feat. Rob Thomas,"October 22, 2006Like seven inches from the mi...",2000
2,2,3,2000,Maria Maria,Santana feat. The Product GandB,Turn up this sound system To the sound of Carl...,2000
3,3,4,2000,I Wanna Know,Joe,"All right, oh, oh, oh It's amazing how you kn...",2000
4,4,5,2000,Everything You Want,Vertical Horizon,Oh and it's rising at the back of your mind Yo...,2000
5,5,6,2000,Say My Name,Destiny's Child,"If no one is around you, say ""Baby I love you""...",2000
6,6,7,2000,I Knew I Loved You,Savage Garden,But some things you just don't question Like i...,2000
7,7,8,2000,Amazed,Lonestar,This feelin' inside me Is almost more than I c...,2000
8,8,9,2000,Bent,matchbox twenty,Pick me up and dust me off And if I get too ti...,2000
9,9,10,2000,He Wasn't Man Enough,Toni Braxton,"Darkchild yeah Listen, girl Who do you think ...",2000


../data/billboard_raw/2001hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,2001,Hanging By A Moment,Lifehouse,Starving for truth I'm closer to where I start...,2000
1,1,2,2001,Fallin',Alicia Keys,In love with you Sometimes I love you Sometime...,2000
2,2,3,2001,All For You,Janet,Shakin' that thing like you never did see Got ...,2000
3,3,4,2001,Drops Of Jupiter (Tell Me),Train,"December 31, 2007With drops of Jupiter in her...",2000
4,4,5,2001,I'm Real,Jennifer Lopez feat. Ja Rule,"Let's go Called you on the phone, said I'm com...",2000
5,5,6,2001,If You're Gone,matchbox twenty,I think you're already gone (gone) I think I'm...,2000
6,6,7,2001,Let Me Blow Ya Mind,Eve feat. Gwen Stefani,"Uh, uh, uh, huh Yo, yo Drop your glasses, shak...",2000
7,7,8,2001,Thank You,Dido,Got out of bed at all The morning rain clouds ...,2000
8,8,9,2001,Again,Lenny Kravitz,I heard a cry within my soul I've never had a ...,2000
9,9,10,2001,Independent Women Part I,Destiny's Child,"Cameron D. and Destiny Charlie's Angels, come ...",2000


../data/billboard_raw/2002hot100.csv


...

,index,position,year,title,artist,lyric,decade
0,0,1,2004,Yeah!,Usher feat. Lil Jon and Ludacris,"Peace up! A town down! [Lil John:] Yeah, yeah,...",2000
1,1,2,2004,Burn,Usher,See it's burning me to hold onto this I know t...,2000
2,2,3,2004,If I Ain't Got You,Alicia Keys,English:Some people live just for the fame S...,2000
3,3,4,2004,This Love,Maroon5,"September 08, 2007The fire burning in her eye...",2000
4,4,5,2004,The Way You Move,OutKast feat. Sleepy Brown,"Ready for action, nip it in the bud We never r...",2000
5,5,6,2004,The Reason,Hoobastank,"June 22, 2007There's many things I wish I did...",2000
6,6,7,2004,I Don't Wanna Know,Mario Winans feat. Enya and P. Diddy,This is another night of these thoughts Can't...,2000
7,7,8,2004,Hey Ya!,OutKast,My baby don't mess around Because she loves me...,2000
8,8,9,2004,Goodies,Ciara feat. Petey Pablo,"My goodies, my goodies My goodies not my goodi...",2000
9,9,10,2004,Lean Back,Terror Squad,"If you know the missing portions, please fill...",2000


../data/billboard_raw/2005hot100.csv


,index,position,year,title,artist,lyric,decade
0,0,1,2005,We Belong Together,Mariah Carey,"Oh, ooh, oh Sweet love, yeah I didn't mean it ...",2000
1,1,2,2005,Hollaback Girl,Gwen Stefani,All the girls stomp your feet like this A few ...,2000
2,2,3,2005,Let Me Love You,Mario,Do you enjoy being hurt? I know you smelled th...,2000
3,3,4,2005,Since U Been Gone,Kelly Clarkson,"It was cool but it was all pretend Yeah, yeah,...",2000
4,4,5,2005,"1, 2 Step",Ciara feat. Missy Elliott,This is a Jazze Phizzle Productshizzle (Jazze ...,2000
5,5,6,2005,Gold Digger,Kanye West feat. Jamie Foxx,She take my money when I'm in need Yeah she's ...,2000
6,6,7,2005,Boulevard of Broken Dreams,Green Day,"December 29, 2006The only one that I have eve...",2000
7,7,8,2005,Candy Shop,50 Cent feat. Olivia,So seductive I'll take you to the candy shop I...,2000
8,8,9,2005,Don't Cha,The Pussycat Dolls feat. Busta Rhymes,"Okay Yeah Oh, we about to get it just a little...",2000
9,9,10,2005,Behind These Hazel Eyes,Kelly Clarkson,"August 16, 2007I used to stand so tall, I use...",2000


../data/billboard_raw/2006hot100.csv


FileNotFoundError: File b'../data/billboard_raw2/2006hot100.csv' does not exist